# ***Byte-Level vs. Token-level Robustness: Comparing Byt5 and T5 under translation of misspellings***

Group: 84  
Students: s252133, s252085, s252086, s251590


## ***Imports***

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## ***Load datasets***

In [3]:
byt5_translated = pd.read_csv("./data/byt5_translated.csv")
byt5_translated.head(3)


,en,es
0,I want to talk to you today a little bit about...,['Hoy quiero hablarles un poco de la irraciona...
1,I was burned very badly.And if you spend a lot...,['Me quemé muy mal.Y si pasan mucho tiempo en ...
2,The nurses in my department thought that the r...,['Las enfermeras en mi departamento pensaban q...


In [4]:
t5_translated = pd.read_csv("./data/t5_translated.csv")
t5_translated.head(3)

,en,es
0,I want to talk to you today a little bit about...,Hoy quiero hablarles un poco sobre la irracion...
1,I was burned very badly.And if you spend a lot...,Me incendió muy mal.Y si pasamos mucho tiempo ...
2,The nurses in my department thought that the r...,Las enfermeras en mi departamento pensaban que...


In [5]:
t5_corrupted = pd.read_csv("./data/t5_corrupt_translated_cleaned.csv")
t5_corrupted.head(3)


,en_corruption_1,es_corruption_1,en_corruption_2,es_corruption_2,en_corruption_3,es_corruption_3
0,I want to talk to you today a little bit about...,Hoy quiero hablarles un poco sobre la irracion...,I whon to talk too your to_day a little bit ab...,Quiero hablar también de la irracionalidad pre...,we mant to talk to you todya a little bit abou...,hace muchos aos hablarles de la irracionalidad...
1,I was burned very badly.And it you spend as lo...,Me incendiéndose muy mal.Y pasas mucho tiempo ...,I was bruned very badley.And is yuo spend an a...,Estaba enfermada muy mal.Y es yuo pasar mucho ...,I saw burned vrey badly.And if you spend the l...,V mal cado.Y si pasas el tiempo una vez si el ...
2,The nurses in my department thought that the r...,Las enfermeras en mi departamento pensaban que...,The nurses in my deportment throught the an ri...,Las enfermeras en mi deportión a través de una...,The nurses it yoy department thougt that the w...,Las enfermeras del departamento tuvieron que e...


In [8]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd

# How many examples to test
N = 10

# Make sure both dataframes are aligned
assert len(t5_translated) >= N and len(t5_corrupted) >= N

# Slice the first N rows
t5_translated_10 = t5_translated.iloc[:N].copy()
t5_corrupted_10  = t5_corrupted.iloc[:N].copy()

# Reference Spanish (gold / target)
refs = t5_translated_10["es"].astype(str).tolist()

# Load multilingual sentence embedding model
model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")

# Encode references once
ref_emb = model.encode(refs, batch_size=64, convert_to_tensor=True)

# Corruption columns you want to evaluate
corruption_cols = ["es_corruption_1", "es_corruption_2", "es_corruption_3"]

for col in corruption_cols:
    # Predicted Spanish for this corruption level
    preds = t5_corrupted_10[col].astype(str).tolist()
    
    # Encode predictions
    pred_emb = model.encode(preds, batch_size=64, convert_to_tensor=True)
    
    # Cosine similarity with corresponding reference
    sims = util.cos_sim(pred_emb, ref_emb).diagonal().cpu().numpy()
    
    # Store similarity in the sliced dataframe
    sim_col = col.replace("es_", "sim_")   # e.g. sim_corruption_1
    t5_corrupted_10[sim_col] = sims

# Optional: define "correct enough" with a threshold
THRESHOLD = 0.85
for col in corruption_cols:
    sim_col = col.replace("es_", "sim_")
    acc_col = col.replace("es_", "acc_")
    t5_corrupted_10[acc_col] = t5_corrupted_10[sim_col] >= THRESHOLD

# Quick look at the first 10 rows for all levels
t5_corrupted_10[[
    "es_corruption_1", "sim_corruption_1", "acc_corruption_1",
    "es_corruption_2", "sim_corruption_2", "acc_corruption_2",
    "es_corruption_3", "sim_corruption_3", "acc_corruption_3",
]]


,es_corruption_1,sim_corruption_1,acc_corruption_1,es_corruption_2,sim_corruption_2,acc_corruption_2,es_corruption_3,sim_corruption_3,acc_corruption_3
0,Hoy quiero hablarles un poco sobre la irracion...,1.000000,True,Quiero hablar también de la irracionalidad pre...,0.944967,True,hace muchos aos hablarles de la irracionalidad...,0.921071,True
1,Me incendiéndose muy mal.Y pasas mucho tiempo ...,0.955024,True,Estaba enfermada muy mal.Y es yuo pasar mucho ...,0.888758,True,V mal cado.Y si pasas el tiempo una vez si el ...,0.782693,False
2,Las enfermeras en mi departamento pensaban que...,0.986206,True,Las enfermeras en mi deportión a través de una...,0.659180,False,Las enfermeras del departamento tuvieron que e...,0.616413,False
3,Y como tena el 70 % de mi cuerpo incendiéndolo...,0.925897,True,"a porque tena el 70 % de mi cuerpo querrió, ll...",0.890106,True,Y en caso de que me incenda el 70 por ciento d...,0.652964,False
4,"Y como pueden imaginar, odiaba ese momento de ...",1.000000,True,"como imaginas, odiaba ese momento de correr co...",0.821959,False,"Y como se puede inmagnia, hice ese momento de ...",0.410001,False
5,"Y tratara de resonar con ellos y decir: ""Por q...",0.933644,True,"Y tratara de resonar ah as que digamos: ""No te...",0.713777,False,"y yo dije: ""Por qué no triamos algo eles?",0.880119,True
6,"Por qué no lo tomamos un poco más tiempo, pued...",0.958482,True,"Por qué no lo tomamos un poco más tiempo, quiz...",1.000000,True,"Por qué no lo ponemos un poco más tiempo, quiz...",0.918628,True
7,las enfermeras me dijeron dos cosas.Me dijeron...,0.951262,True,Y las enfermeras me dijeron dos cosas.Me dijer...,0.979229,True,Y las enfermeras me dijeron dos cosas.thrar te...,0.873300,True
8,As que eso fue lo que hice.,1.000000,True,As que eso fue lo que hice.,1.000000,True,As que vat es lo que hice.,0.955312,True
9,Todava estaba interesado en esta queston de có...,0.859765,True,Todava estaba interesado en esta pregunta de c...,0.910596,True,Todava estaba interesado en la pregunta de cóm...,0.634895,False
